# Part 1 pre-acceptance workflow

This notebook is meant to be a prototype for the NVIDIA NIM hackathon project.

## Installation and Requirements

pls run with gpu kernel or !pip installs for 3.10 in python3 kernel, while the notebook is using 3.9.
Also, unfortunate horrible inconsistency between some package (maybe pdyplumber) which needs numpy 2, and pandas which refuses numpy 2 and only does 1.26.4. If shit go wrong in the future in an import, you know what to check

In [ ]:
# Requirements
%pip install langchain==0.2.5
%pip install langchain_community==0.2.5
%pip install faiss-gpu # replace with faiss-gpu if you are using GPU
%pip install faiss-cpu
%pip install langchain-nvidia-ai-endpoints==0.1.2
%pip install requests pdfplumber spacy camelot-py 
%pip install pandas==2 numpy==1.26.4 
%pip install beautifulsoup4 
%pip install pymupdf
%pip install lxml
%pip install unstructured

: 

## LLM & Embedding

### 1) Initialize the LLM

The ChatNVIDIA class is part of LangChain's integration (langchain_nvidia_ai_endpoints) with NVIDIA NIM microservices.
It allows access to NVIDIA NIM for chat applications, connecting to hosted or locally-deployed microservices.

Here we will use **mixtral-8x7b-instruct-v0.1**

Note: You can use any model hosted at the NVIDIA API catalog using 'ChatNVIDIA.get_available_models()'

#### Note:
- In this notebook, we have used NVIDIA NIM microservices from the NVIDIA API Catalog.
- The other APIs, ChatNVIDIA, NVIDIAEmbedding, and NVIDIARerank, also support self-hosted NIM microservices.
- Change the `base_url` to your deployed NIM URL.
- Example: `llm = ChatNVIDIA(base_url="http://localhost:8000/v1", model="meta/llama3-8b-instruct")`
- NIM can be also hosted locally using Docker, following the [NVIDIA NIM for LLMs](https://docs.nvidia.com/nim/large-language-models/latest/getting-started.html) documentation. This is only true if you are the son of a rich oil tycoon, and have a few H100s sitting around in your basement.

In [11]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# connect to an LLM NIM running at localhost:8000, specifying a specific model
llm = ChatNVIDIA(base_url="http://localhost:8000/v1", model="meta/llama3-8b-instruct")

### 2) Intialize the embedding
NVIDIAEmbeddings is a client to NVIDIA embeddings models that provides access to a NVIDIA NIM for embedding. It can connect to a hosted NIM or a local NIM using a base URL

We selected **NV-Embed-QA** as the embedding

In [12]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="nvidia/nv-embedqa-e5-v5", base_url="http://localhost:8001/v1", truncate="END")

### 3) Obtain dataset
I love taxes and work! Lets steal information about taxes and work!

#### In the **DataHandler** class defined below, we can;

A) Walk through a webpage and find all sub-webpages and scrape the parent and children,

B) Extract texts, tables and images from pdfs

C) Split documents 

- Real world documents can be very long, this makes it hard to fit in the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

- To handle this we’ll split the Document into chunks for embedding and vector storage. More on text splitting [here](https://python.langchain.com/v0.2/docs/concepts/#text-splitters)

In [13]:
import os
import requests
import pandas as pd
import urllib.parse  # To handle URL joining
import fitz

import numpy as np
from tqdm import tqdm
from io import StringIO
from bs4 import BeautifulSoup, SoupStrainer
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, DataFrameLoader, CSVLoader, UnstructuredTSVLoader

class DataHandler:
    """
    Masterfully handles data scraping, preprocessing, and other data-related functionalities in this notebook.
    """

    def __init__(self, embedder, csv_path="./data/csv", pdf_path="./data/pdf"):
        # Paths and storage
        self.csv_path = csv_path
        self.pdf_path = pdf_path
        self.visited_urls = set()

        # Data containers
        self.raw_data = []
        self.all_data = []
        self.textual_data = []
        self.textual_metadata = []
        self.tabular_data = []
        self.tabular_metadata = []
        self.visual_data = []
        self.visual_metadata = []
        
        # Final data containers
        self.embedded_data = []
        self.metadata = []

        # Text splitter
        self.text_splitter = CharacterTextSplitter(chunk_size=2048, separator=" ", chunk_overlap=64)

        # Ensure directories exist
        os.makedirs(self.csv_path, exist_ok=True)
        os.makedirs(self.pdf_path, exist_ok=True)
        
        self.embedder = embedder
        
    @staticmethod
    def clean_text(text):
        """
        Cleans text to reduce storage and improve readability
        
        Args:
            - text (str):   text to clean
        """
        return " ".join(text.split())
    
    
    def filter_text(self, text, similarity_threshold=0.5):
        """
        Filters through mass amounts of data to see if there is relevance to our use case

        Args:
            - text (str):   text to check and filter through
        """
        keywords = [ "law", "compliance", "regulation", "tax", "contract", "finance", "employment", "business", "corporate", "trade", "policy", "license", "permits", "registration", "audit", "liability", "shareholders", "partnership", "incorporation", "startup", "profit", "revenue", "penalty", "customs", "dispute", "governance", "authority", "import", "export", "data", "privacy", "director", "management", "intellectual", "property", "trademark", "compliance", "ownership", "capital", "dividend", "funding", "duty", "penalty", "fine", "subsidiary", "merger", "acquisition", "bankruptcy", "insolvency"]

        def cosine_similarity(vec1, vec2):
            """
            Computes the cosine similarity between two vectors and returns cosine similarity score

            Args:
                - vec1: First embedding vector
                - vec2: Second embedding vector
            """
            return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)) 
    
        words = set(text.lower().split())
        matches = set(keywords).intersection(words)
        keyword_score = len(matches) / len(words) if len(words) > 0 else 0
        
        if len(text) > 0:
            query_embedding = self.embedder.embed_query("Relevant and important information about corporate laws, compliance requirements, tax obligations, and regulatory guidelines for Small and Medium Enterprises (SMEs) operating in Singapore.")
            text_embedding = self.embedder.embed_query(text)
            similarity_score = cosine_similarity(query_embedding, text_embedding)
        else:
            similarity_score = 0

        return (0.1 * keyword_score) + (0.9 * similarity_score) > 0.5
    

    def embed_text(self, text):
        """
        Embeds a given text if necessary

        Args:
            - text (str):   text to embed
        """
        return self.embedder.embed_query(text)
    
    @staticmethod
    def read_csv(csv_path: str) -> CSVLoader:
        """
        Reads a CSV file and returns a LangChain CSVLoader instance.

        Args:
            - csv_path (str):   path to csv file
        """
        df = pd.read_csv(csv_path)
        return CSVLoader(
            file_path=csv_path,
            csv_args={"delimiter": ",", "quotechar": '"', "fieldnames": df.columns.tolist()}
        )
    
    def scrape_csvs(self, directory: str):
        """
        Adds CSV files from a directory to the tabular data loaders.

        Args:
            - directory (str):  directory of csv files
        """
        for root, _, files in os.walk(directory):
            for file in files:
                if file.endswith(".csv"):
                    file_path = os.path.join(root, file)
                    loader = self.read_csv(file_path)
                    for row in loader.load()[1:]:
                        if self.filter_text(row.page_content):
                            split_data = self.text_splitter.split_text(self.clean_text(row.page_content))

                            self.tabular_data.extend(split_data)
                            self.tabular_metadata.extend([{"source": file, "date_added": "None"} for _ in split_data])
                            print(f"Current csv: {file}")


    def extract_table_elements(self, url, tables):
        """
        Extracts tabular data from HTML tables and saves them as CSV and loaders

        Args:
            - url (str):        url the table is from
            - tables (list):    list of table elements
        """
        for idx, table in enumerate(tables):
            try:
                # Clean up table superscripts
                for tag in table.find_all("sup"):
                    tag.extract()

                # Extract the table name
                header = table.find_previous(["h3", "h4"])
                table_name = self.clean_text(header.text) if header and header.text else f"table_{idx}"
                table_name = os.path.basename(url) + f" {table_name}"

                # Parse table into a DataFrame
                df = pd.read_html(StringIO(str(table)), header=0)[0]
                df["context"] = table_name
                for index, row in df.iterrows():
                    row_context = [f"Row {idx+1} - {col}: {str(row[col])}" for col in df.columns]
                    full_context = " | ".join(row_context)  # Join all columns with a separator to keep context
                    if self.filter_text(full_context):
                        split_data = self.text_splitter.split_text(self.clean_text(full_context))

                        self.tabular_data.extend(split_data)
                        self.tabular_metadata.extend([{"source": url, "date_added": "None"} for _ in split_data])
                        print(f"Current table: {table_name}")

            except Exception as e:
                print(f"Failed to extract table from {url}: {e}")

    def create_loaders(self):
        """
        Processes raw data to create LangChain loaders

        Args:   None
        """
        for url, soup in self.raw_data:
            main_content = soup.find("main") 
            if main_content:
                raw_text = main_content.get_text(separator=" ", strip=True)
                if self.filter_text(raw_text):
                    split_data = self.text_splitter.split_text(self.clean_text(raw_text))

                    self.textual_data.extend(split_data)
                    self.textual_metadata.extend([{"source": url, "date_added": "None"} for _ in split_data])

    def scrape_websites(self, urls, max_depth, depth=0):
        """
        Recursively scrapes a website for HTML content and tables and then processes the data into loaders

        Args:
            - urls (list):      list of urls to scrape from
            - max_depth (int):  how many sublinks into the main website we wish to mdig through
            - depth (int):      current/starting depth of sublinks reached
        """
        for base_url in urls:
            def _scrape(url, depth):
                if url in self.visited_urls or depth > max_depth:
                    return

                try:
                    response = requests.get(url)
                    if response.status_code != 200:
                        print(f"Failed to retrieve {url}")
                        return

                    soup = BeautifulSoup(response.content, "html.parser")
                    self.visited_urls.add(url)
                    self.raw_data.append((url, soup))

                    # Process tables
                    self.extract_table_elements(url, soup.find_all("table"))

                    # Recurse through links
                    for link in soup.find_all("a", href=True):
                        abs_url = urllib.parse.urljoin(url, link["href"])
                        if base_url in abs_url:
                            _scrape(abs_url, depth + 1)
                    print(f"Current url: {url}")

                except Exception as e:
                    print(f"Error accessing {url}: {e}")

            _scrape(base_url, depth)
            self.create_loaders()

    def scrape_pdfs(self, pdf_folder=None):
        """
        Extracts text from PDF files.
        Args:
            - pdf_folder (str): folder of pdf files
        """
        pdf_folder = pdf_folder or self.pdf_path
        for file in os.listdir(pdf_folder):
            if file.endswith(".pdf"):
                pdf_path = os.path.join(pdf_folder, file)
                try:
                    doc = fitz.open(pdf_path)
                    pdf_text = "".join(page.get_text("text") for page in doc)
                    if self.filter_text(pdf_text):
                        split_text = self.text_splitter.split_text(self.clean_text(pdf_text))

                        self.textual_data.extend(split_text)
                        self.textual_metadata.extend([{"source": pdf_path, "date_added": "None"} for _ in split_text])
                        print(f"Current pdf: {pdf_path}")

                except Exception as e:
                    print(f"Failed to process {pdf_path}: {e}")
                
    def prepare_data_for_insertion(self):
        """
        Prepares data before data can be added into the vector db

        Args:   None
        """
        self.all_data = self.textual_data + self.tabular_data
        self.embedded_data = [self.embed_text(text) for text in tqdm(self.textual_data, desc=f"Total pieces of textual data: {len(self.textual_data)}")] + [self.embed_text(text) for text in tqdm(self.tabular_data, desc=f"Total pieces of tabular data: {len(self.tabular_data)}")]
        self.metadata = self.textual_metadata + self.tabular_metadata

USER_AGENT environment variable not set, consider setting it to identify your requests.


### We define our websites, pdfs and csvs below to call the dataloader on.

In [ ]:
websites = [
    # "https://www.iras.gov.sg",
    "https://www.mom.gov.sg",
    "https://www.acra.gov.sg",
    "https://singaporelegaladvice.com",
    "https://www.ipos.gov.sg",
    "https://www.enterprisesg.gov.sg",
    "https://www.skillsfuture.gov.sg",
    "https://www.hsa.gov.sg",
    "https://www.sfa.gov.sg"
]

datahandler = DataHandler(embedder=embedder)
datahandler.scrape_websites(websites, max_depth=5)
datahandler.scrape_pdfs(r'sgp_bootcamp_material/project/data/pdfs')
datahandler.scrape_csvs(r'sgp_bootcamp_material/project/data/csvs')

Current url: https://www.mom.gov.sg/about-us#MainContent
Current url: https://www.mom.gov.sg/passes-and-permits
Current url: https://www.mom.gov.sg/employment-practices
Current url: https://www.mom.gov.sg/workplace-safety-and-health
Current url: https://www.mom.gov.sg/eservices
Current url: https://www.mom.gov.sg/eservices/calculators
Current url: https://www.mom.gov.sg/forms
Current url: https://www.mom.gov.sg/newsroom
Current url: https://www.mom.gov.sg/eservices/services/mymom-portal
Current url: https://www.mom.gov.sg/about-us#SearchMOM
Current url: https://www.mom.gov.sg/about-us/vision-mission-and-values
Current url: https://www.mom.gov.sg/about-us/divisions-and-statutory-boards
Current url: https://www.mom.gov.sg/about-us/logo
Current url: https://www.mom.gov.sg/about-us/accolades
Current url: https://www.mom.gov.sg/about-us/cos2024
Current url: https://www.mom.gov.sg/about-us/cos2023
Current url: https://www.mom.gov.sg/about-us/cos2022
Current url: https://www.mom.gov.sg/about-

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Current url: https://www.acra.gov.sg/docs/default-source/default-document-library/how-to-guides/preparing-financial-statements/practice-guidance-1-of-2024---areas-of-review-focus-for-fy2024-financial-statements.pdf
Current url: https://www.acra.gov.sg/news-events/news-details/id/829
Failed to extract table from https://www.acra.gov.sg/news-events/news-details/id/824: No tables found matching pattern '.+'
Failed to extract table from https://www.acra.gov.sg/news-events/news-details/id/824#pgTop: No tables found matching pattern '.+'
Current url: https://www.acra.gov.sg/news-events/news-details/id/824#pgTop


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Current url: https://www.acra.gov.sg/docs/default-source/news-events-documents/2024/list-of-bizfile-eservices-that-are-unavailable.pdf
Failed to extract table from https://www.acra.gov.sg/how-to-guides/buying-information/our-partners: No tables found matching pattern '.+'
Current url: https://www.acra.gov.sg/how-to-guides/buying-information/our-partners
Failed to extract table from https://www.acra.gov.sg/about-bizfile: No tables found matching pattern '.+'
Current url: https://www.acra.gov.sg/about-bizfile
Failed to extract table from https://www.acra.gov.sg/about-bizfile/resources: No tables found matching pattern '.+'
Current url: https://www.acra.gov.sg/about-bizfile/resources
Current url: https://www.acra.gov.sg/news-events/news-details/id/824
Failed to extract table from https://www.acra.gov.sg/news-events/news-details/id/827: No tables found matching pattern '.+'
Failed to extract table from https://www.acra.gov.sg/news-events/news-details/id/827#pgTop: No tables found matching 

...

Current url: https://www.sfa.gov.sg/docs/default-source/digital-services/payments/payment-faqs-and-guides.pdf
Current url: https://www.sfa.gov.sg/digital-services/payments/axs
Current url: https://www.sfa.gov.sg/digital-services/payments/online-payment


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Current url: https://www.sfa.gov.sg/docs/default-source/e-service/payment-modes/payment-faqs-and-guides.pdf
Failed to retrieve https://www.sfa.gov.sg/docs/default-source/digital-services/food-import-and-export/giro-form-with-bre-sfa


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Current url: https://www.sfa.gov.sg/docs/default-source/digital-services/payment-faqs-and-guides.pdf
Current url: https://www.sfa.gov.sg/tools-and-resources/e-giro
Current url: https://www.sfa.gov.sg/digital-services/payments/giro
Current url: https://www.sfa.gov.sg/digital-services/payments/gstrefund
Current url: https://www.sfa.gov.sg


### After creating all the loaders, we prepare the data to be inserted into the vector database.

In [65]:
# placeholder global variables incase you fuckup and want to edit the class but dont want to lose the variables you generated 

placeholder_textual_data = datahandler.textual_data 
placeholder_textual_metadata = datahandler.textual_metadata 
placeholder_tabular_data = datahandler.tabular_data 
placeholder_tabular_metadata = datahandler.tabular_metadata 


In [ ]:
len(datahandler.textual_data)

In [68]:
datahandler = DataHandler(embedder=embedder)
datahandler.textual_data = placeholder_textual_data
datahandler.textual_metadata = placeholder_textual_metadata
datahandler.tabular_data = placeholder_tabular_data
datahandler.tabular_metadata = placeholder_tabular_metadata

In [69]:
datahandler.prepare_data_for_insertion()




Total pieces of textual data: 12009:   0%|          | 0/12009 [00:00<?, ?it/s]


Total pieces of textual data: 12009:   0%|          | 5/12009 [00:00<04:42, 42.47it/s]


Total pieces of textual data: 12009:   0%|          | 11/12009 [00:00<04:03, 49.22it/s]


Total pieces of textual data: 12009:   0%|          | 16/12009 [00:00<04:10, 47.83it/s]


Total pieces of textual data: 12009:   0%|          | 22/12009 [00:00<04:01, 49.57it/s]


Total pieces of textual data: 12009:   0%|          | 28/12009 [00:00<03:57, 50.43it/s]


Total pieces of textual data: 12009:   0%|          | 34/12009 [00:00<03:49, 52.15it/s]


Total pieces of textual data: 12009:   0%|          | 40/12009 [00:00<03:43, 53.45it/s]


Total pieces of textual data: 12009:   0%|          | 47/12009 [00:00<03:34, 55.66it/s]


Total pieces of textual data: 12009:   0%|          | 53/12009 [00:01<03:32, 56.30it/s]


Total pieces of textual data: 12009:   0%|          | 60/12009 [00:01<03:27, 57.57it/s]


Total pieces of t

### 4) Storing the documents

To build our foundational knowledge base from our collected data and allow for faster retrieval of vector queries, we need to have some form of search system. To do this, we can implement hybridrag, which combines  GraphRAG and VectorRAG approaches to create a more thorough, complete retrieval system. 

#### a) Store data into a graphdb, thus building a knowledge graph of the data we have

The obvious choice here is neo4j, which has integration with langchain to recieve LLM interpretations of documents.

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

In [3]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost"
AUTH = ("neo4j", "cringemfpassword") # (cringe admin username and pw)

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity() #vett connection, if shit dont work gg buddy

The above can only be run when a neo4j instance is active. If it is not, up the docker compose to run it and the milvus DB service.

In [4]:
graph = Neo4jGraph(
    url="neo4j://localhost:7687",
    username="neo4j",
    password="cringemfpassword",
)

#### b) Store all our data into a vectordb (connects vector stores to structured db systems)

We will be using **Milvus DB** since it is an open-source, distributed vector DB designed for high-performance vector similarity search across massive datasets (which is what we kind of will have), and makes use of popular libraries like FAISS or Annoy for its vector searching.

We have the option to use;
1) MilvusVectorStore (Milvus Lite) which is easier to implement and can be more easily tied with ML frameworks --> Milvus-lite does not have a windows version (only supports linux and mac)
2) MilvusClient (Python SDK) WITHOUT Docker which was used in a demo to build a RAG system --> Milvus-lite does not have a windows version (only supports linux and mac)
3) Direct DB WITH Docker (Milvus Standalone) to directly manage and control the connections as well as increase flexibility --> What im using first since it is the most straighforward (im trying to find if milvus standalone can be used locally instead of through docker)

Sources used:
- https://milvus.io/docs/quickstart.md
- https://milvus.io/docs/integrate_with_langchain.md
- https://milvus.io/docs/multimodal_rag_with_milvus.md
- https://github.com/milvus-io/bootcamp/tree/master/bootcamp/tutorials/quickstart/apps/multimodal_rag_with_milvus (Possible demo to use)

##### Note:
Previously, I thought of using **Weaviate** because it is a free, open-source, scalable and reliable vector database service with decent amounts of documentation online.

The Weaviate DB works more with Object-Oriented design rather than rows/columns and does not need to be structured like an SQL DB since it already has integrated models/pre-computed embeddings to handle vector embeddings.

However, I realised it **could not** be GPU accelerated during its vector retrieval/indexing.

Sources used:
- https://python.langchain.com/docs/integrations/vectorstores/weaviate/
- https://weaviate.io/developers/weaviate/client-libraries/python

In [ ]:
%pip install langchain-milvus
%pip install -U pymilvus

Defaulting to user installation because normal site-packages is not writeable
  Using cached setuptools-75.5.0-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 11.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 6.8 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 8.8 MB/s eta 0:00:00ta 0:00:01
Using cached setuptools-75.5.0-py3-none-any.whl (1.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, 

In [5]:
from pymilvus import MilvusClient, DataType, CollectionSchema, FieldSchema
import hashlib

class MilvusDB:
    """
    Not-so-masterfully handles the vector database using Milvus and all DB related functionality.
    """

    def __init__(self, uri):
        self.uri = uri
        self.client = None
        self.collection_name = None

        self.similarity_threshold = 0.99  # Similarity between new input data and existing data
        self.set_up_db()

    def set_up_db(self):
        """
        Starts up connection to Weaviate DB and creates schema if not already created

        Parameters: None

        Output:     None
        """
        self.client = MilvusClient(uri=self.uri)

    def load_collection(self):
        """
        Loads collection into RAM for faster retrieval

        Parameters: None

        Output:     None
        """
        self.client.load_collection(
            collection_name=self.collection_name,
            replica_number=1 # Number of replicas to create on query nodes. Max value is 1 for Milvus Standalone, and no greater than `queryNode.replicas` for Milvus Cluster.
        )

    def release_collection(self):
        """
        Releases the collection from memory to save memory usage

        Parameters: None

        Output:     None
        """
        self.client.release_collection(
            collection_name=self.collection_name
        )

    def create_collection(self, collection_name, dimensions):
        """
        Creates a new collection in the DB

        Parameters: 
            - collection_name:  Name of collection to make
            - dimensions:       Number of dimensions for vector data

        Output:     None
        """     
        # Defines a schema to follow when creating the DB
        id_field = FieldSchema(name="id", dtype=DataType.VARCHAR, max_length=512, is_primary=True)
        embedding_field = FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dimensions)
        text_field = FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=2500)  # Use VARCHAR for string types

        # Metadata
        source_field = FieldSchema(name="source", dtype=DataType.VARCHAR, max_length=512)  
        date_added_field = FieldSchema(name="date_added", dtype=DataType.VARCHAR, max_length=50)
        category_field = FieldSchema(name="category", dtype=DataType.VARCHAR, max_length=100)
        relevancy_field = FieldSchema(name="relevancy", dtype=DataType.FLOAT)

        schema = CollectionSchema(fields=[id_field, embedding_field, text_field, source_field, date_added_field, category_field, relevancy_field])

        # Creates the collection
        self.client.create_collection(
            collection_name=collection_name,
            dimension=dimensions,
            schema=schema,
            metric_type="COSINE",             
            consistency_level="Strong",     
        )
        self.collection_name = collection_name

        # Creates an index for more efficient similarity search later on based on the metric_type and index_type
        self.index_params = MilvusClient.prepare_index_params()
        self.index_params.add_index(
            field_name="embedding",
            metric_type="COSINE",
            index_type="IVF_FLAT",
            index_name="embedding_index",
            params={ "nlist": 128 }
        )
        self.client.create_index(
            collection_name=self.collection_name,
            index_params=self.index_params,
            sync=False
        )

    def insert_data(self, original, embedded, metadata=None, batch_size=128):
        """
        Adds document and embedding object pairs to the DB collection if not alreadt inside 

        Parameters:
            - original:     Original documents
            - embedded:     Embedded documents
            - metadata:     List of dictionaries containing metadata for each document
            - batch_size:   Number of records per batch

        Output: None
        """
        self.load_collection()
        
        data_batch = []
        print(len(original), len(embedded), len(metadata))
        for i, embedded_line in enumerate(tqdm(embedded, desc="Inserting data...")):
            unique_id = self.generate_unique_id(embedded_line)            
            if self.check_for_similar_vectors(embedded_line) and self.check_for_similar_ids(unique_id):  # Update existing document if ID is found
                self.update_existing_document(unique_id, embedded_line, original[i])
            else:
                categories = self.generate_data_categories(original[i])
                data_batch.append(
                    {
                        "id": unique_id, 
                        "embedding": embedded_line, 
                        "text": original[i], 
                        "source": metadata[i].get("source", "unknown") if metadata else 'None',
                        "date_added": metadata[i].get("date_added", "unknown") if metadata else 'None',
                        "category": categories,
                        "relevancy": 1.0
                    }
                )

            if len(data_batch) >= batch_size:  # Inserts data batch when exceeding max batch size
                self.client.insert(collection_name=self.collection_name, data=data_batch)
                data_batch = []

        if data_batch:
            self.client.insert(collection_name=self.collection_name, data=data_batch)

        self.release_collection()

    def update_existing_document(self, doc_id, embedding, text, relevancy=1.0):
        """
        Updates an existing document in the collection by replacing its embedding and text

        Parameters:
            - doc_id:       Unique ID of the document
            - embedding:    New embedding vector for the document
            - text:         New text for the document

        Output: None
        """
        self.client.delete(collection_name=self.collection_name, filter=f"id == '{doc_id}'")
        
        updated_data = [{"id": doc_id, "embedding": embedding, "text": text, "source": "None", "date_added":'None', "category":"None", "relevancy":relevancy}]
        self.client.insert(collection_name=self.collection_name, data=updated_data)

    def update_document_scores(self, doc_id, feedback):
        """
        Updates the relevancy score of certain pieces of data based on feedback from users

        Parameters:
            - doc_id:       Unique ID of the document
            - feedback:     Feedback given by user

        Output: None
        """
        print(self.client.search(collection_name=self.collection_name, filter=f"id == '{doc_id}'", output_fields=["text", "source", "date_added", "category", "relevancy"])[0])
        embeddding, text, score = self.client.search(collection_name=self.collection_name, filter=f"id == '{doc_id}'", output_fields=["text", "source", "date_added", "category", "relevancy"])[0]
        
        # Update score based on feedback
        if feedback == "Very Relevant": score += 0.2
        elif feedback == "Not Relevant": score -= 0.1 
        else: score += 0.1

        self.update_existing_document(doc_id, embeddding, text, score)

    def generate_data_categories(self, text):
        """
        Generates tags associated with a piece of text of question

        Parameters:
            - data:         Text data to categorise

        Returns:
            - categories:   List of categories relevant to data
        """
        categories = llm.invoke(f" \
            You are an expert in Singapore's corporate laws and regulations. Your task is to analyze the given text or question, and assign it appropriate tags from the list below. Tags should capture the core topics, entities, and relevant categories mentioned in the text or question given. \
            Only return 3 words of the the top 3 most relevant tags from the list below, without giving any other comments. \
            List of Tags: \
            - Sole Proprietorship, Partnership, Company, Corporation \
            - Business Registration, Taxation, Licensing, Employment Laws, Compliance, Intellectual Property, Contracts \
            - Setup, Operations, Reporting, Closure \
            - Healthcare, Retail, Finance, Technology, Food \
            - Forms, Policies, Guidelines, FAQs \
            - Compliance Issues, Tax Problems, Employment Disputes, Licensing Delays \
            - Singapore, International Trade \
            Input Text: {text} \
            Output Format: Comma-separated list of tags \
        ").content.strip().lower()
        categories = min(categories[0:len(categories)],  categories[0:100])
        return categories

    def generate_unique_id(self, data):
        """
        Generates a unique hash ID based on the vector or text data

        Parameters:
            - data: The vector or text data used to generate the hash

        Returns:
            - id:   A unique hash ID as a string
        """
        data_str = str(data)
        unique_id = hashlib.sha256(data_str.encode()).hexdigest()
        return unique_id

    def check_for_similar_vectors(self, embedding, top_k=5):
        """
        Checks the DB for vectors that are similar to the input embedding (based on distance metric like cosine similarity or Euclidean distance)

        Parameters:
            - embedding:    Embedded documents
            - top_k:        Top K number of documents that are similar to the input embedding

        Output: 
            - check:        True or False value of whether a similar vector has been found
        """
        try:
            search_params = {"metric_type": "COSINE", "params": {"nprobe": 10}}
            results = self.client.search(
                collection_name=self.collection_name,
                data=[embedding],
                anns_field="embedding",  # Adjust field name based on your Milvus schema
                search_params=search_params,
                limit=top_k
            )
            for result in results:
                for vector in result:
                    if vector['distance'] >= self.similarity_threshold:
                        return True  
            return False
        except Exception as e:
            print(f"Error checking if vector exists: {e}")
            return False

    def check_for_similar_ids(self, id):
        """
        Checks the DB for ids that are similar to the new one created

        Parameters:
            - id:       Unique id generated for new row

        Output: 
            - check:    True or False value of whether a similar vector has been found
        """
        try:
            results = self.client.query(
                collection_name=self.collection_name,
                filter=f"id == '{id}'",  
                output_fields=["id"], limit=1000 
            )
            return True if results else False
        except Exception as e:
            print(f"Error checking if ID exists: {e}")
            return False

    def retrieve_data(self, embedded_question, question, top_k=5, scaling_factor=1.1):
        """
        Retrieves vector data from DB based on embedded question

        Parameters:
            - question:             Question as a string
            - embedded_question:    Embedded question as a vector

        Output:
            - search_res:           Results of vector retrieval
        """
        self.load_collection()

        category_filter = " or ".join([f'category like "%{category}%"' for category in self.generate_data_categories(question).replace(' ', '').split(',')])

        search_res = self.client.search(
            collection_name=self.collection_name,
            data=[embedded_question],  
            limit=top_k, 
            search_params={"metric_type": "COSINE", "params": {"nprobe": 10}},
            filter=category_filter, 
            output_fields=["text", "source", "date_added", "category"]
        )[0]

        distances = [res['distance'] for res in search_res]
        avg_distance = sum(distances) / len(distances) if distances else 0
        adaptive_threshold = avg_distance * scaling_factor  # Getting a dynamic threshold based on average distance of top-K results to improve relevance filtering

        search_res = self.client.search(
            collection_name=self.collection_name,
            data=[embedded_question],  
            limit=20, 
            search_params={"metric_type": "COSINE", "params": {"nprobe": 10}},
            filter=category_filter, 
            output_fields=["text", "source", "date_added", "category", "relevancy"]
        )[0]

        filtered_res = [res for res in search_res if res['distance'] > adaptive_threshold]
        self.release_collection()
        return filtered_res
    
    def get_all_records(self, limit=10000):
        """
        Retrieves all rows from the Milvus collection.
        
        Parameters:
            - limit: The maximum number of rows to fetch in one query. Adjust as needed.
        
        Returns:
            - List of all records in the collection.
        """
        self.load_collection()

        try:
            # Query all records
            all_records = self.client.query(
                collection_name=self.collection_name,
                output_fields=["id", "text", "source", "date_added", "category"],
                limit=limit  # Adjust limit as needed
            )
            return all_records
        except Exception as e:
            print(f"Error retrieving records: {e}")
            return []
        finally:
            self.release_collection()

In [8]:
database = MilvusDB('http://localhost:19530')
database.collection_name = 'Documents'


In [ ]:
database = MilvusDB('http://localhost:19530')
database.create_collection("Documents", 1024)
database.insert_data(datahandler.textual_data + datahandler.tabular_data, datahandler.embedded_data, datahandler.metadata)

12153 12153 12153





Inserting data...:   0%|          | 0/12153 [00:00<?, ?it/s]


Inserting data...:   0%|          | 1/12153 [00:00<1:50:50,  1.83it/s]

business registration, taxation, compliance <class 'str'>





Inserting data...:   0%|          | 2/12153 [00:00<1:27:17,  2.32it/s]

taxation, compliance, singapore <class 'str'>


...




Inserting data...:  53%|█████▎    | 6403/12153 [1:01:03<42:01,  2.28it/s]

company, intellectual property, compliance <class 'str'>





Inserting data...:  53%|█████▎    | 6404/12153 [1:01:03<41:09,  2.33it/s]

singapore, compliance, intellectual property <class 'str'>





Inserting data...:  53%|█████▎    | 6438/12153 [1:01:23<1:02:18,  1.53it/s]


Inserting data...:  53%|█████▎    | 6439/12153 [1:01:24<1:00:47,  1.57it/s]


Inserting data...:  53%|█████▎    | 6440/12153 [1:01:25<59:38,  1.60it/s]  


Inserting data...:  53%|█████▎    | 6441/12153 [1:01:25<53:11,  1.79it/s]


Inserting data...:  53%|█████▎    | 6442/12153 [1:01:25<48:38,  1.96it/s]


Inserting data...:  53%|█████▎    | 6443/12153 [1:01:26<51:09,  1.86it/s]


Inserting data...:  53%|█████▎    | 6444/12153 [1:01:27<52:57,  1.80it/s]


Inserting data...:  53%|█████▎    | 6445/12153 [1:01:27<48:28,  1.96it/s]


Inserting data...:  53%|█████▎    | 6446/12153 [1:01:27<45:20,  2.10it/s]


Inserting data...:  53%|█████▎    | 6447/12153 [1:01:28<54:31,  1.74it/s]


Inserting data...:  53%|█████▎    | 6448/12153 [1:01:29<55:18,  1.72it/s]


Inserting data...:  53%|█████▎    | 6449/12153 [1:01:30<1:01:33,  1.54it/s]


Inserting data...:  53%|█████▎    | 6450/12153 [1:01:30<1:00:10,  1.58it/s]



In [71]:
database.client.drop_collection("Documents")

In [9]:
records = database.get_all_records()
print(len(records))
# for record in records: print(record)

484


### 5) Using data to answer questions

With our stored embedded data, we can retrieve relevant vectors stored in our vectorstore/db to answer embedded questions.

#### A) Using vectorstore to answer

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

context = store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

# Langchain's LCEL(LangChain Expression Language) Runnable protocol is used to define the chain
# LCEL allows pipe together components and functions
chain = (
    {"context": context, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### B) Using VectorDB to answer

What happens if our vectordb does not have the relevant information needed though?

#### C) Using web sources to answer

Using Tavily, we can create a search engine that calls abstract searching, scraping, filtering and extracting from online sources.

Only issue is, it takes too long to get the answer out, so it should only be used as a final resort.

In [98]:
import os
os.environ["TAVILY_API_KEY"] = "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"

In [99]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState
from typing import Literal

class SearchEngine:
    def __init__(self):
        self.search_tool = TavilySearchResults(max_results=3)
        self.llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

        self.tools = [self.search_tool]  # Add other tools here if needed
        self.llm_with_tools = self.llm.bind_tools(self.tools)
        self.workflow = self.build_workflow()

    def build_workflow(self):
        """Builds the state graph workflow."""
        workflow = StateGraph(MessagesState)
        
        # Add nodes for agent (LLM) and tools (search)
        workflow.add_node("agent", self.call_model)
        workflow.add_node("tools", ToolNode(self.tools))
        
        # Add conditional logic to determine if tools should be used
        workflow.add_edge("__start__", "agent")
        workflow.add_conditional_edges("agent", self.should_continue)
        workflow.add_edge("tools", "agent")
        
        return workflow.compile()

    def call_model(self, state):
        """Invoke the LLM with the current state."""
        messages = state["messages"]
        response = self.llm_with_tools.invoke(messages)
        return {"messages": [response]}

    def should_continue(self, state):
        """Determine whether to continue invoking tools or end the workflow."""
        messages = state["messages"]
        last_message = messages[-1]
        if last_message.tool_calls:
            return "tools"
        return "__end__"

    def query(self, user_query, context=' in singapore'):
        """Main method to process the user's query."""
        initial_state = MessagesState({"messages": [self.llm_with_tools.invoke(user_query + context)]})
        return self.workflow.invoke(initial_state)

In [142]:
def ask(question):
    relevance = llm.invoke(f" \
        Context from database:\n\n\n\
        User's question: {question}\n\n\
        Is the question relevant to navigating Legal and Regulatory Requirements in Singapore's Corporate Landscape, involving taxes, employment, costs and such? \
        Reply only either a 'yes' or 'no'.\
    ").content.strip().lower()
    
    if 'yes' in relevance:
        retrieved_data = database.retrieve_data(embedder.embed_query(question), question)

        if len(retrieved_data) > 0:
            retrieved_lines_with_distances = [
                (res["entity"]["text"], res["distance"], res["entity"]["source"]) for res in retrieved_data
            ]
            context = '\n'.join(
                [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
            )
            prompt = f" \
                Context from database:\n{context}\n\n\
                User's question: {question}\n\n\
                Please generate an informative response and cite the source after.\
                If the question is irrelevant to SME Regulations, reply that that the question is irrelevant.\
            "
            return llm.invoke(prompt).content
        
        print("No data found in database. Searching online...")
        search_engine = SearchEngine()
        response = search_engine.query(question)
        print(response)
        for message in response['messages']:
            print(message.content)
            database.insert_data(message.content, embedder.embed_query(message.content))
        return
    else:
        return "The question is irrelevant to SME Regulations."
    

#### Now, we can try asking different types of questions:

Case 1: Irrelevant question

In [11]:
print(chain.invoke("What is component two hundred and twenty five million?"))

There is no information provided in the documents about a component or entity called "two hundred and twenty five million." The documents contain information about various financial transactions, mortgage and loan balances, estate duty calculations, and rules related to M&A allowance and motor vehicle expenses, but there is no mention of a "component" with a value of 225 million.


In [53]:
print(ask("What is component two hundred and twenty five million?"))

The question is irrelevant to SME Regulations.


Case 2: Simple questions

In [ ]:
print(chain.invoke("How do i file taxes for my company?"))

In [143]:
print(ask("How do i file taxes for my company?"))

company, taxation, reporting, singapore <class 'str'>
category like "%company%" or category like "%taxation%" or category like "%reporting%" or category like "%singapore%"
[]
No data found in database. Searching online...


ValidationError: 1 validation error for TavilySearchAPIWrapper
__root__
  Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. (type=value_error)

Case 3: Complex questions

In [ ]:
print(chain.invoke("In the event my foriegn employee is injured at work, how do i report the incident and claim reparations?"))

Case 4: Realistic questions

In [13]:
chain.invoke("I am an employer whose firm qualifies for PWCs. From 2025 onwards, how much Co-Funding can I recieve from the government?")
# In its current state, the model does an ethan for this quite hard question, although it at least does retrieve some (few?) relevant PWCS info.

'From 2025 onwards, as an employer whose firm qualifies for PWCS (Productivity Works Credits Scheme), you can receive co-funding from the government of up to 50% for the first tier of wage increases and 15% to 30% for the second tier of wage increases. This co-funding support applies to wage increases given in qualifying year 2025 and onwards. The gross monthly wage ceiling for PWCS co-funding will be increased to $3,000 in qualifying years 2025 and 2026.\n\nPlease note that the specific rates and details of the co-funding support may be subject to changes or updates in the PWCS guidelines. It is recommended to consult the official guidelines or contact the relevant authorities for the most accurate and up-to-date information.'

### 6) Enhancing accuracy for single data sources

This example demonstrates how a re-ranking model can be used to combine retrieval results and improve accuracy during retrieval of documents.

Typically, reranking is a critical piece of high-accuracy, efficient retrieval pipelines. Generally, there are two important use cases:

- Combining results from multiple data sources
- Enhancing accuracy for single data sources

Here, we focus on demonstrating only the second use case. If you want to know more, check [here](https://github.com/langchain-ai/langchain-nvidia/blob/main/libs/ai-endpoints/docs/retrievers/nvidia_rerank.ipynb)

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIARerank
from langchain_core.runnables import RunnableParallel

# We will narrow the collection to 100 results and further narrow it to 10 with the reranker.
retriever = store.as_retriever(search_kwargs={'k':100}) # typically k will be 1000 for real world use-cases
ranker = NVIDIARerank(model='nv-rerank-qa-mistral-4b:1', top_n=10)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

reranker = lambda input: ranker.compress_documents(query=input['question'], documents=input['context'])

chain_with_ranker = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | {"context": reranker, "question": lambda input: input['question']}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
print(chain_with_ranker.invoke("How do i file taxes for my company?"))

Based on the provided documents, to file taxes for your company, you need to follow these steps:

1. Ensure that you are duly authorized by your company as an 'Approver' for Corporate Tax (Filing and Applications) in Corppass. You can refer to the step-by-step guides for assistance on Corppass setup.
2. Have your Singpass and your company’s Unique Entity Number (UEN)/ Entity ID ready.
3. Visit the mytax.iras.gov.sg website to file the Corporate Income Tax Return for your company.
4. If your company is filing Form C, you need to submit the financial statements/certified accounts and tax computation(s) for the relevant Year of Assessment (YA).
5. If your company meets the qualifying conditions to file Form C-S or Form C-S (Lite), you can choose to file the simplified version, Form C-S (Lite), if your company has an annual revenue of $200,000 or below.

You can also visit the Basic Guide to Corporate Income Tax for Companies page to get help with filing your company’s tax returns for the 

In [ ]:
print(chain_with_ranker.invoke("In the event my foriegn employee is injured at work, how do i report the incident and claim reparations?"))

Based on the provided documents, if your foreign employee is injured at work, you can report the incident and claim reparations under the Work Injury Compensation Act (WICA). Specifically, the documents mention that input tax can be claimed for work injury compensation insurance that is obligatory under WICA for both local and foreign employees performing manual work or non-manual work earning $2,600 or less a month.

To report the incident and make a claim, you can visit the Ministry of Manpower (MOM) webpage on WICA or contact MOM at +65 6438 5122. However, it is important to note that medical and accident insurance premiums for your staff are generally not allowable for input tax claims under the GST (General) Regulations, unless the insurance or payment of compensation is obligatory under WICA or under any collective agreement within the meaning of the Industrial Relations Act.

Therefore, it seems that you can report the incident and claim reparations under WICA, but you should ve